In [1]:
%%bash
chia version
cdv --version
python --version

1.5.2.dev2601
cdv, version 1.1.1
Python 3.10.6


# Set Up Chia Simulator

In [3]:
%%bash
chia show -s
chia keys show

Network: simulator0    Port: 43858   RPC Port: 23141
Node ID: 5d4086cbb6613b4105aab8e9785a582ba309d87ff97824025438acca801fac16
Genesis Challenge: eb8c4d20b322be8d9fddbf9412016bdffe9a2901d7edb0e364e94266d0e095f7
Current Blockchain Status: Full Node Synced

Peak: Hash: 4564db5decced36090f38e3e29a1a1aea1fc6331b6fa42f08287af15391e4bc6
      Time: Tue Sep 20 2022 22:22:56 +07                  Height:          2

Estimated network space: 7.030 MiB
Current difficulty: 1024
Current VDF sub_slot_iters: 1024

  Height: |   Hash:
        2 | 4564db5decced36090f38e3e29a1a1aea1fc6331b6fa42f08287af15391e4bc6
        1 | 9bade947f87384e17cae7f08df08d769fcb0e3d58bc72007f9a0f1dd82eb7593
Showing all public keys derived from your master seed and private key:

Label: main
Fingerprint: 3570018204
Master public key (m): b986e494e60434e38002d9ebe528107d693237ea7f188abd03d629f905211a8f3b195548bf92493f0d0133f09ec3a82e
Farmer public key (m/12381/8444/0/0): 95d4be2f8cf70e444bdc9568598a21e0fdc7690b9ca4f34371b9d9b

In [4]:
%%bash
# farm to Alice's address
cdv sim farm --target-address 'txch1x9r36u9tryfum2uz2qdejjfvcgmlk9nwcjd9qlhqegqqqlldyq6s4ul5wl'
# farm to Bob's address
cdv sim farm --target-address 'txch1lf6qhdld63ryf9uyh32l6ycfjegx5vmuquvpr77l2z2ez4wu2nzscxalh4'

Farmed 1 Transaction blocks
Block Height is now: 3
Farmed 1 Transaction blocks
Block Height is now: 4


In [9]:
%%bash
chia wallet show -f 2620633628

Wallet height: 6
Sync status: Synced
Balances, fingerprint: 2620633628

Chia Wallet:
   -Total Balance:         2.0 txch (2000000000000 mojo)
   -Pending Total Balance: 2.0 txch (2000000000000 mojo)
   -Spendable:             2.0 txch (2000000000000 mojo)
   -Type:                  STANDARD_WALLET
   -Wallet ID:             1
 
Connections:
Type      IP                                     Ports       NodeID      Last Connect      MiB Up|Dwn
FULL_NODE 127.0.0.1                              43858/43858 5d4086cb... Sep 20 22:24:38      0.0|0.1    
                                                 -Height: No Info    -Hash: No Info    -Trusted: True


In [11]:
%%bash
chia wallet show -f 505650069

Wallet height: 6
Sync status: Synced
Balances, fingerprint: 505650069

Chia Wallet:
   -Total Balance:         2.0 txch (2000000000000 mojo)
   -Pending Total Balance: 2.0 txch (2000000000000 mojo)
   -Spendable:             2.0 txch (2000000000000 mojo)
   -Type:                  STANDARD_WALLET
   -Wallet ID:             1
 
Connections:
Type      IP                                     Ports       NodeID      Last Connect      MiB Up|Dwn
FULL_NODE 127.0.0.1                              43858/43858 5d4086cb... Sep 20 22:24:44      0.0|0.1    
                                                 -Height: No Info    -Hash: No Info    -Trusted: True


In [12]:
%%bash
cdv decode txch1x9r36u9tryfum2uz2qdejjfvcgmlk9nwcjd9qlhqegqqqlldyq6s4ul5wl

31471d70ab1913cdab82501b99492cc237fb166ec49a507ee0ca00007fed2035


In [13]:
%%bash
cdv rpc coinrecords --by puzzlehash 31471d70ab1913cdab82501b99492cc237fb166ec49a507ee0ca00007fed2035 -nd

{
    "29a287174bdbbad83dc48f95b45100f583f2ac29675af528475ce61133546cac": {
        "coin": {
            "amount": 1750000000000,
            "parent_coin_info": "0xeb8c4d20b322be8d9fddbf9412016bdf00000000000000000000000000000003",
            "puzzle_hash": "0x31471d70ab1913cdab82501b99492cc237fb166ec49a507ee0ca00007fed2035"
        },
        "coinbase": true,
        "confirmed_block_index": 4,
        "spent_block_index": 0,
        "timestamp": 1663687441
    },
    "354ef4bfed5c2a127d6989f8333b279e6af37079a4affa2a2f8c45f3a63982c8": {
        "coin": {
            "amount": 250000000000,
            "parent_coin_info": "0xfe9a2901d7edb0e364e94266d0e095f700000000000000000000000000000003",
            "puzzle_hash": "0x31471d70ab1913cdab82501b99492cc237fb166ec49a507ee0ca00007fed2035"
        },
        "coinbase": true,
        "confirmed_block_index": 4,
        "spent_block_index": 0,
        "timestamp": 1663687441
    }
}


# Prepare Public Keys from Alice (2620633628) and Bob (505650069)

In [14]:
%%bash
chia keys derive -f 2620633628 child-key -t wallet -i 0 --show-hd-path
chia keys derive -f 505650069 child-key -t wallet -i 0 --show-hd-path

Wallet public key 0 (m/12381/8444/2/0): b9b03d861f5781e4611f3be8f1147249e5efbbf9cf41e7abc8325a7a2ba97f4f8789718af8d222efbbe3680ec000b9db
Wallet public key 0 (m/12381/8444/2/0): b808ef948e5d09284543dca71921508a5e2b30e954fde583dfc4e99de22b63a51f547ec937d6f24aa7f6f9bb5995c149


In [31]:
# chia libraries
from blspy import (PrivateKey, AugSchemeMPL, G1Element, G2Element)
from chia.types.blockchain_format.program import Program
from chia.wallet.puzzles import (p2_delegated_puzzle_or_hidden_puzzle)
# utils & tic tac toe helper code
import sys
sys.path.insert(0, "../code")
from utils import (load_program, print_program, print_puzzle, print_json)
import singleton_utils
import tic_tac_toe

# puzzles
tic_tac_toe_puzzle = load_program("../code/tic-tac-toe.clsp", ["../code"])
coin_puzzle = load_program("../code/coin.clsp", ["../code"])
terminate_puzzle = load_program("../code/terminate-game.clsp", ["../code"])

waiting_room_puzzle = load_program("../code/waiting-room.clsp", ["../code"])
play_amount = 1_000_000_000_000

In [16]:
alice_pk = G1Element.from_bytes(bytes.fromhex("b9b03d861f5781e4611f3be8f1147249e5efbbf9cf41e7abc8325a7a2ba97f4f8789718af8d222efbbe3680ec000b9db"))
alice_puzzle_hash = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_pk(alice_pk).get_tree_hash()
bob_pk = G1Element.from_bytes(bytes.fromhex("b808ef948e5d09284543dca71921508a5e2b30e954fde583dfc4e99de22b63a51f547ec937d6f24aa7f6f9bb5995c149"))
bob_puzzle_hash = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_pk(bob_pk).get_tree_hash()

# prepare players info
player_one_info = Program.to([alice_pk, alice_puzzle_hash])
player_two_info = Program.to([bob_pk, bob_puzzle_hash])


## Create Waiting Room Coins
[waiting-room.clsp](../code/waiting-room.clsp)
```clojure
(mod (OWNER_PUZZLE_HASH PK action)
```

In [17]:
from chia.util.bech32m import encode_puzzle_hash
## alice
alice_waiting_room_puzzle = waiting_room_puzzle.curry(
    alice_puzzle_hash,
    alice_pk
)
alice_waiting_room_puzzle_hash = alice_waiting_room_puzzle.get_tree_hash()
alice_waiting_room_address = encode_puzzle_hash(alice_waiting_room_puzzle_hash, "txch")
alice_waiting_room_address

'txch19f0wmlraav4xgpuvggz7vuhj52ma6enj55eapuhylxth4x3f7v6sx9tdwp'

In [18]:
## bob
bob_waiting_room_puzzle = waiting_room_puzzle.curry(
    bob_puzzle_hash,
    bob_pk
)
bob_waiting_room_puzzle_hash = bob_waiting_room_puzzle.get_tree_hash()
bob_waiting_room_address = encode_puzzle_hash(bob_waiting_room_puzzle_hash, "txch")
bob_waiting_room_address

'txch1q5gkykpez0whmz8at39ddn6l766aafuualt2t5e9v8aprheghjgqurtcqc'

In [20]:
%%bash
chia wallet send -f 2620633628 -t txch19f0wmlraav4xgpuvggz7vuhj52ma6enj55eapuhylxth4x3f7v6sx9tdwp -a 1.000000000001

Submitting transaction...
Transaction submitted to nodes: [{'peer_id': '5d4086cbb6613b4105aab8e9785a582ba309d87ff97824025438acca801fac16', 'inclusion_status': 'SUCCESS', 'error_msg': None}]
Run 'chia wallet get_transaction -f 2620633628 -tx 0x4023478f440ee231a28ac67671ac479aff85dfe8bcdb1bf97b4e13f6c7bc6a42' to get status


In [23]:
%%bash
chia wallet send -f 505650069 -t txch1q5gkykpez0whmz8at39ddn6l766aafuualt2t5e9v8aprheghjgqurtcqc -a 1

Submitting transaction...
Transaction submitted to nodes: [{'peer_id': '5d4086cbb6613b4105aab8e9785a582ba309d87ff97824025438acca801fac16', 'inclusion_status': 'SUCCESS', 'error_msg': None}]
Run 'chia wallet get_transaction -f 505650069 -tx 0x3bb44a7936ee4025cd977b97fbf841eb627e0208931573765e36d7b3f9413358' to get status


## Terminate Puzzle (Curried)
[terminate-game.clsp](../code/terminate-game.clsp)
```clojure
(mod (IS_SINGLETON PLAYER_ONE_HASH PLAYER_TWO_HASH P2_AMOUNT play_result)
```

In [24]:
player_amount = 1_000_000_000_000
player_fee = 50_000_000
p2_amount = player_amount - player_fee
game_amount = (player_amount * 2) + 1 # odd amount for singleton
curried_terminate_puzzle = terminate_puzzle.curry(
    1, # Terminate Singleton Coin
    alice_puzzle_hash,
    bob_puzzle_hash,
    p2_amount
)

## Coin Puzzle (Curried)
[coin.clsp](../code/coin.clsp)
```clojure
(mod (MOD TERMINATE_PUZZLE PLAYER_ONE_INFO PLAYER_TWO_INFO CURRIED_TIC_TAC_TOE_PUZZLE AMOUNT position)
```

In [25]:
def get_coin_puzzle(board, player):
    # (mod (BOARD V pos)
    curried_tic_tac_toe_puzzle = tic_tac_toe_puzzle.curry(
            Program.to(board), 
            Program.to(player)
        ) 

    #(mod (MOD PLAYER_ONE_INFO PLAYER_TWO_INFO CURRIED_TIC_TAC_TOE_PUZZLE amount position)
    curried_coin_puzzle = coin_puzzle.curry(
        coin_puzzle,
        curried_terminate_puzzle,
        player_one_info,
        player_two_info,
        curried_tic_tac_toe_puzzle,
        game_amount)
    return curried_coin_puzzle

curried_coin_puzzle = get_coin_puzzle([' '] * 9, 'x')

## Launcher Coin

In [28]:
from chia.rpc.full_node_rpc_client import FullNodeRpcClient
from chia.util.config import load_config
from chia.util.default_root import DEFAULT_ROOT_PATH
from chia.util.ints import uint16

config = load_config(DEFAULT_ROOT_PATH, "config.yaml")
selected_network = config["selected_network"]
genesis_challenge = config["farmer"]["network_overrides"]["constants"][selected_network]["GENESIS_CHALLENGE"]

self_hostname = config["self_hostname"] # localhost
full_node_rpc_port = config["full_node"]["rpc_port"] # 8555
full_node_client = await FullNodeRpcClient.create(
    self_hostname, uint16(full_node_rpc_port), DEFAULT_ROOT_PATH, config
)
coin_records = await full_node_client.get_coin_records_by_puzzle_hash(alice_waiting_room_puzzle_hash)
full_node_client.close()
await full_node_client.await_closed()

alice_waiting_room_puzzle_coin = coin_records[0].coin

In [29]:
from chia.types.blockchain_format.coin import Coin
from chia.types.coin_spend import CoinSpend
from chia.wallet.puzzles import singleton_top_layer_v1_1

# prepare launcher coin
launcher_coin = Coin(
    alice_waiting_room_puzzle_coin.name(), # waiting_room's coin spend creates the launcher coin
    singleton_top_layer_v1_1.SINGLETON_LAUNCHER_HASH, 
    game_amount
)

launcher_id = launcher_coin.name()
print(f'\nlauncher id: {launcher_id}')

singleton_struct = (
    singleton_top_layer_v1_1.SINGLETON_MOD_HASH, 
    (launcher_id, singleton_top_layer_v1_1.SINGLETON_LAUNCHER_HASH)
)

singleton_puzzle = singleton_top_layer_v1_1.SINGLETON_MOD.curry(
    singleton_struct,
    curried_coin_puzzle, # tic tac toe coin puzzle
)
launcher_coin


launcher id: b'\xcf\xef\xe5%\xa9\x80\xe0\x81\xb5\x98D\xbb\x91B8\x02s\x15\xf3g?\xeb\x7f\x98>\xcfbK\x10\x10\xbd\x1f'


Coin { parent_coin_info: f4b89c7651bfe282c6292ef2cf890e9e479abe8475ae772d15df9160df5a0984, puzzle_hash: eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9, amount: 2000000000001 }

In [32]:
launcher_solution = Program.to(
    [
        singleton_puzzle.get_tree_hash(),
        game_amount,
        [
            ("game", "tic tac toe"), 
            ("player one", alice_puzzle_hash),
            ("player two", bob_puzzle_hash),
        ]
    ]
)
launcher_announcement = launcher_solution.get_tree_hash()

launcher_coin_spend = CoinSpend(
    launcher_coin,
    singleton_top_layer_v1_1.SINGLETON_LAUNCHER,
    launcher_solution
)
print_json(launcher_coin_spend.to_json_dict())

{
    "coin": {
        "amount": 2000000000001,
        "parent_coin_info": "0xf4b89c7651bfe282c6292ef2cf890e9e479abe8475ae772d15df9160df5a0984",
        "puzzle_hash": "0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9"
    },
    "puzzle_reveal": "0xff02ffff01ff04ffff04ff04ffff04ff05ffff04ff0bff80808080ffff04ffff04ff0affff04ffff02ff0effff04ff02ffff04ffff04ff05ffff04ff0bffff04ff17ff80808080ff80808080ff808080ff808080ffff04ffff01ff33ff3cff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff0effff04ff02ffff04ff09ff80808080ffff02ff0effff04ff02ffff04ff0dff8080808080ffff01ff0bffff0101ff058080ff0180ff018080",
    "solution": "0xffa0e68b47f242a438c856daf32f0caa0977974dda5517bc3d75df012b6e4ced5723ff8601d1a94a2001ffffff8467616d658b7469632074616320746f65ffff8a706c61796572206f6e65a031471d70ab1913cdab82501b99492cc237fb166ec49a507ee0ca00007fed2035ffff8a706c617965722074776fa0fa740bb7edd446449784bc55fd130996506a337c071811fbdf50959155dc54c58080"
}
